In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("energydata_complete.csv")

In [3]:
df.head(7)

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097
5,2016-01-11 17:50:00,50,40,19.89,46.026667,19.2,44.500000,19.79,44.933333,18.890000,...,17.000000,45.29,6.016667,734.0,92.0,5.333333,43.833333,4.8,44.919484,44.919484
6,2016-01-11 18:00:00,60,50,19.89,45.766667,19.2,44.500000,19.79,44.900000,18.890000,...,17.000000,45.29,5.900000,734.1,92.0,5.000000,40.000000,4.7,47.233763,47.233763


In [4]:
df.columns

Index(['date', 'Appliances', 'lights', 'T1', 'RH_1', 'T2', 'RH_2', 'T3',
       'RH_3', 'T4', 'RH_4', 'T5', 'RH_5', 'T6', 'RH_6', 'T7', 'RH_7', 'T8',
       'RH_8', 'T9', 'RH_9', 'T_out', 'Press_mm_hg', 'RH_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1', 'rv2'],
      dtype='object')

In [5]:
df.isnull().sum()

date           0
Appliances     0
lights         0
T1             0
RH_1           0
T2             0
RH_2           0
T3             0
RH_3           0
T4             0
RH_4           0
T5             0
RH_5           0
T6             0
RH_6           0
T7             0
RH_7           0
T8             0
RH_8           0
T9             0
RH_9           0
T_out          0
Press_mm_hg    0
RH_out         0
Windspeed      0
Visibility     0
Tdewpoint      0
rv1            0
rv2            0
dtype: int64

In [6]:
df = df.drop(['date'], axis=1)

## To build the model

In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df2 = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
X = df2.drop(['lights'], axis=1)
y = df2.Appliances


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=42)

In [9]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
predicted_values = linear_model.predict(X_test)


## To measure our Regression Model's perfomance

In [10]:
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, predicted_values)
mae
#round(mae, 3)

1.3935207081787437e-15

In [11]:
rss = np.sum(np.square(y_test - predicted_values))
rss
#round(rss, 2)

3.253941419574648e-26

In [12]:
from sklearn.metrics import mean_squared_error
mse = np.sqrt(mean_squared_error(y_test, predicted_values))
mse

#(mse, 3)

2.3442684057809147e-15

In [13]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predicted_values)
#round(r2, 3)
r2

1.0

In [14]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(X_train, y_train)
predicted_values = ridge_reg.predict(X_test)

In [15]:
from sklearn.linear_model import Lasso
Lasso_reg = Lasso(alpha=0.001)
Lasso_reg.fit(X_train, y_train)
predicted_values = Lasso_reg.predict(X_test)

## To compare the effects of Regularization

In [16]:
def get_weights_df(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ["Features", col_name]
    weights_df[col_name].round(3)
    return weights_df


linear_model_weights = get_weights_df(linear_model, X_train, 'Linear_model_weights')
ridge_weights_df = get_weights_df(ridge_reg, X_train, 'Ridge_Weights')
lasso_weights_df = get_weights_df(Lasso_reg, X_train, 'Lasso_Weights')

final_weights = pd.merge(linear_model_weights, ridge_weights_df, on='Features')
final_weights2 = pd.merge(final_weights, lasso_weights_df, on='Features')

print(final_weights2)


       Features  Linear_model_weights  Ridge_Weights  Lasso_Weights
0          RH_2         -1.116015e-15      -0.001493      -0.000000
1            T1         -6.017356e-16      -0.000067       0.000000
2            T7         -5.491956e-16       0.000037       0.000000
3            T6         -4.103113e-16       0.000789       0.000000
4            T2         -4.007695e-16      -0.000732       0.000000
5          RH_4         -3.681769e-16       0.000089       0.000000
6          RH_8         -3.331592e-16      -0.000570      -0.000000
7   Press_mm_hg         -2.263818e-16       0.000024      -0.000000
8    Visibility         -2.177389e-16       0.000044       0.000000
9            T5         -1.601474e-16      -0.000072       0.000000
10         RH_6         -7.279523e-17       0.000129      -0.000000
11          rv1         -2.413313e-17       0.000003      -0.000000
12          rv2         -2.413313e-17       0.000003      -0.000000
13         RH_5         -5.743978e-18       0.00